<a href="https://colab.research.google.com/github/lekiho/GTA1/blob/main/GTA_wadi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
my_path = '/content/notebooks'

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab_Notebooks/GTA/myenv')
sys.path.append('/content/drive/MyDrive/Colab_Notebooks/GTA')

In [ ]:
print(sys.path)

['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/Colab_Notebooks/GTA/myenv', '/content/drive/MyDrive/Colab_Notebooks/GTA']


In [ ]:
# !pip install --target=$my_path -r /content/drive/MyDrive/'Colab Notebooks'/GTA/requirements.txt

In [ ]:
def down_cast(df, columns) :
    df_list = []
    for column in columns :
        try:
            df[column] = pd.to_numeric(df[column], downcast='integer')
        except:
            pass
        try:
            df[column] = pd.to_numeric(df[column], downcast='float')
        except:
            pass
        try:
            df[column] = pd.to_object(df[column], downcast='object')
        except:
            pass
        df_list.append(df[column])
    return pd.concat(df_list, axis=1)

In [ ]:
import os

import pandas as pd
import numpy as np
#downsampling
from myenv import process_wadi
import re
from sklearn.preprocessing import MinMaxScaler

In [ ]:
dirpath = os.getcwd()
x_train = pd.read_csv(os.path.join(dirpath, 'data', '/content/drive/MyDrive/Colab_Notebooks/GTA/data/WADI.A2_19_Nov_2019/WADI_14days_new.csv'), index_col=[0])

In [ ]:
x_test = pd.read_csv(os.path.join(dirpath, 'data', '/content/drive/MyDrive/Colab_Notebooks/GTA/data/WADI.A2_19_Nov_2019/WADI_attackdataLABLE.csv'), index_col=[0])

In [ ]:
down_cast(x_train, x_train.columns)
down_cast(x_test, x_test.columns)

In [ ]:
x_test['Attack LABLE (1:No Attack, -1:Attack)'] = (x_test['Attack LABLE (1:No Attack, -1:Attack)'] -1)*((-1)/2)
x_test['attack'] = x_test['Attack LABLE (1:No Attack, -1:Attack)'].astype(int)
y_test = x_test['attack']
x_test.drop('Attack LABLE (1:No Attack, -1:Attack)', axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  


In [ ]:
# x_train.to_csv('/content/drive/MyDrive/Colab Notebooks/GTA/data/WADI.A2_19_Nov_2019/WADI_14days_train.csv')
# x_test.to_csv('/content/drive/MyDrive/Colab Notebooks/GTA/data/WADI.A2_19_Nov_2019/WADI_attackdata_labelled_test.csv')

##### process_wadi

In [ ]:
# max min(0-1)
def norm(train, test):

    normalizer = MinMaxScaler(feature_range=(0, 1)).fit(train) # scale training data to [0,1] range
    train_ret = normalizer.transform(train)
    test_ret = normalizer.transform(test)

    return train_ret, test_ret

In [ ]:
# downsample by 10
def downsample(data, labels, down_len):
    np_data = np.array(data)
    np_labels = np.array(labels)

    orig_len, col_num = np_data.shape

    down_time_len = orig_len // down_len

    np_data = np_data.transpose()
    # print('before downsample', np_data.shape)

    d_data = np_data[:, :down_time_len*down_len].reshape(col_num, -1, down_len)
    d_data = np.median(d_data, axis=2).reshape(col_num, -1)

    d_labels = np_labels[:down_time_len*down_len].reshape(-1, down_len)
    # if exist anomalies, then this sample is abnormal
    d_labels = np.round(np.max(d_labels, axis=1))

    d_data = d_data.transpose()

    # print('after downsample', d_data.shape, d_labels.shape)

    return d_data.tolist(), d_labels.tolist()

In [ ]:
train = x_train
test = x_test

train = train.iloc[:, 2:]
test = test.iloc[:, 2:]


train = train.fillna(train.mean())
test = test.fillna(test.mean())
train = train.fillna(0)
test = test.fillna(0)

# trim column names
train = train.rename(columns=lambda x: x.strip())
test = test.rename(columns=lambda x: x.strip())


train_labels = np.zeros(len(train))
test_labels = test.attack

# train = train.drop(columns=['attack'])
test = test.drop(columns=['attack'])

cols = [x[46:] for x in train.columns] # remove column name prefixes
train.columns = cols
cols = [x[46:] for x in test.columns] # remove column name prefixes
test.columns = cols


x_train1, x_test1 = norm(train.values, test.values)

train = pd.DataFrame(x_train1)
test = pd.DataFrame(x_test1)

# for i, col in enumerate(train.columns):
#     train.loc[:, col] = x_train1[:, i]
#     test.loc[:, col] = x_test1[:, i]

d_train_x, d_train_labels = downsample(train.values, train_labels, 10)
d_test_x, d_test_labels = downsample(test.values, test_labels, 10)

train_df = pd.DataFrame(d_train_x, columns = train.columns)
test_df = pd.DataFrame(d_test_x, columns = test.columns)


test_df['attack'] = d_test_labels
train_df['attack'] = d_train_labels

# train_df = train_df.iloc[2160:]

In [ ]:
train_df.to_csv('/content/drive/MyDrive/Colab_Notebooks/GTA/data/WADI.A2_19_Nov_2019/WADI_14days_downsampled.csv')

##### GTA

In [22]:
# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.11.0+cu113.html
     |████████████████████████████████| 7.9 MB 14.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.11.0+cu113.html
     |████████████████████████████████| 3.5 MB 14.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.11.0+cu113.html
     |████████████████████████████████| 2.5 MB 14.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.11.0+cu113.html
     |████████████████████████████████| 750 kB 14.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.d

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
my_path = '/content/notebooks'
sys.path.append('/content/drive/MyDrive/Colab_Notebooks/GTA/myenv')
sys.path.append('/content/drive/MyDrive/Colab_Notebooks/GTA')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ! pip install --target=$my_path -r /content/drive/MyDrive/Colab_Notebooks/GTA/requirements.txt

In [ ]:
%cd /content/drive/MyDrive/Colab_Notebooks/GTA
import myenv
# from exp import exp_gta_dad
# from models import gta

/content/drive/MyDrive/Colab_Notebooks/GTA


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
x_train = pd.read_csv(os.path.join('data', '/content/drive/MyDrive/Colab_Notebooks/GTA/data/rename/WADI_14days_colab.csv'), index_col=[0])

In [ ]:
# x_train['date'] = pd.to_datetime(x_train[['date','Time']].apply(lambda row:' '.join(row.values.astype(str)), axis=1))

In [ ]:
# x_train['date'] = pd.to_datetime(x_train['date'], format = '%y-%m-%d %H:%M:%S')

In [ ]:
pd.DataFrame(x_train.isnull().sum())

In [ ]:
x_train.drop(['2_LS_001_AL','2_LS_002_AL','2_P_001_STATUS','2_P_002_STATUS'], axis=1, inplace=True)

In [ ]:
x_train.dropna(axis=0,inplace=True)

In [ ]:
x_train.reset_index(inplace=True,drop=True)

In [ ]:
pd.DataFrame(x_train.isnull().sum())

In [ ]:
# from datetime import datetime
# import time
# x_train['timestamp'] = x_train['date']
# for i in range(len(x_train['date'])):
#   x_train['timestamp'].iloc[i] = time.mktime(x_train['date'].iloc[i].timetuple())

In [ ]:
# x_train.to_csv('/content/drive/MyDrive/Colab_Notebooks/GTA/data/rename/WADI_14days_colab.csv')

In [130]:
attack_data = pd.read_csv(os.path.join('data', '/content/drive/MyDrive/Colab_Notebooks/GTA/data/rename/WADI_attackdata_colab.csv'), index_col=[0])

In [ ]:
pd.DataFrame(attack_data.isnull().sum())

In [ ]:
attack_data.drop(['2_LS_001_AL','2_LS_002_AL','2_P_001_STATUS','2_P_002_STATUS'], axis=1, inplace=True)

In [ ]:
attack_data.reset_index(inplace=True, drop=True)

In [ ]:
# attack_data.drop([172801,172802], axis=0, inplace=True)

In [ ]:
# attack_data['Timestamp'] = pd.to_datetime(attack_data[['Date ', 'Time']].apply(lambda row:' '.join(row.values.astype(str)), axis=1))

In [ ]:
# attack_data.drop(['Date ','Time'], axis=1, inplace = True)

In [ ]:
# from datetime import datetime
# import time
# for i in range(len(attack_data['Timestamp'])):
#   attack_data['Timestamp'].iloc[i] = time.mktime(attack_data['Timestamp'].iloc[i].timetuple())

In [ ]:
# attack_data['label'] = attack_data['Attack LABLE (1:No Attack, -1:Attack)']
# attack_data.drop(['Attack LABLE (1:No Attack, -1:Attack)'], axis=1, inplace = True)

In [ ]:
# attack_data.to_csv('/content/drive/MyDrive/Colab_Notebooks/GTA/data/rename/WADI_attackdata_colab.csv')

In [ ]:
%cd /content/drive/MyDrive/Colab_Notebooks/GTA

/content/drive/MyDrive/Colab_Notebooks/GTA


In [ ]:
!python -u main_gta_dad.py --model gta --data WADI --root_path '/content/drive/MyDrive/Colab_Notebooks/GTA/data/rename' --data_path '/data/rename/WADI_14days_colab.csv'

True
Use GPU: cuda:0
>>>>>>>start training : gta_WADI_ftM_sl60_ll30_pl1_nl3_dm124_nh8_el3_dl2_df124_atprob_ebfixed_test_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 784477
val 43140
test 172741
	iters: 100, epoch: 1 | loss: 12148.6167065
	speed: 0.1949s/iter; left time: 4566.0963s
	iters: 200, epoch: 1 | loss: 11994.8482236
	speed: 0.1921s/iter; left time: 4483.0497s
	iters: 300, epoch: 1 | loss: 11841.0826446
	speed: 0.1919s/iter; left time: 4458.5808s
	iters: 400, epoch: 1 | loss: 11687.3179089
	speed: 0.1907s/iter; left time: 4412.5628s
	iters: 500, epoch: 1 | loss: 11533.5552690
	speed: 0.1909s/iter; left time: 4396.2788s
	iters: 600, epoch: 1 | loss: 11379.7933792
	speed: 0.1900s/iter; left time: 4356.4497s
	iters: 700, epoch: 1 | loss: 11226.0312035
	speed: 0.1922s/iter; left time: 4387.3917s
	iters: 800, epoch: 1 | loss: 11072.2691530
	speed: 0.1930s/iter; left time: 4388.2136s
	iters: 900, epoch: 1 | loss: 10918.5079703
	speed: 0.1953s/iter; left time: 4420.3225s
	iters: 1000, epoch: 1 | 

In [225]:
import numpy as np
import pandas as pd
label = np.load('/content/drive/MyDrive/Colab_Notebooks/GTA/results/gta_WADI_ftM_sl60_ll30_pl1_nl3_dm124_nh8_el3_dl2_df124_atprob_ebfixed_test_0/label.npy')
metrics = np.load('/content/drive/MyDrive/Colab_Notebooks/GTA/results/gta_WADI_ftM_sl60_ll30_pl1_nl3_dm124_nh8_el3_dl2_df124_atprob_ebfixed_test_0/metrics.npy')
pred =  np.load('/content/drive/MyDrive/Colab_Notebooks/GTA/results/gta_WADI_ftM_sl60_ll30_pl1_nl3_dm124_nh8_el3_dl2_df124_atprob_ebfixed_test_0/pred.npy')
true =  np.load('/content/drive/MyDrive/Colab_Notebooks/GTA/results/gta_WADI_ftM_sl60_ll30_pl1_nl3_dm124_nh8_el3_dl2_df124_atprob_ebfixed_test_0/true.npy')

In [288]:
import utils.spot as spot

In [251]:
f = pred -true

In [255]:
for i in range(len(f)):
  f[i] = ((pred[i] -true[i])**2).sum()

In [263]:
f = f.squeeze()
# f = torch.from_numpy(f)
mp = nn.MaxPool1d(124)
f = mp(f)

In [266]:
f = f.squeeze()

In [302]:
s = spot.SPOT()

In [303]:
s.fit(f[:1000], f[1000:])

In [304]:
s.initialize()

Initial threshold : 2.889427731608381
Number of peaks : 19
Grimshaw maximum log-likelihood estimation ... [done]
	γ = 0
	σ = 0.442265015342376
	L = 3.4989261071700293
Extreme quantile (probability = 0.0001): 5.210002913384255


In [349]:
f

array([2.24127476, 2.24085309, 2.32034758, ..., 4.18156693, 4.1226509 ,
       4.11103465])

In [395]:
f1 = np.where(f > 4.4, -1 , 1)

In [396]:
f2 = np.append(f1, np.ones(201))

In [397]:
pd.DataFrame(f2).value_counts()

 1.0    124225
-1.0     48576
dtype: int64

In [398]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
print(f1_score(attack_data['label'], f2))
print(recall_score(attack_data['label'], f2))
print(confusion_matrix(attack_data['label'], f2))

0.8150942870381014
0.7184813049673267
[[  2738   7239]
 [ 45838 116986]]


In [401]:
pd.DataFrame(f2).to_csv( '/content/drive/MyDrive/Colab_Notebooks/GTA/results/wadi_gta_se.csv')

In [400]:
pd.DataFrame(f2).value_counts()

 1.0    124225
-1.0     48576
dtype: int64

In [ ]:
attack_data['label'].value_counts()

 1    162824
-1      9977
Name: label, dtype: int64

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
print(f1_score(attack_data['label'], label))
print(recall_score(attack_data['label'], label))
print(confusion_matrix(attack_data['label'], label))

0.9948410553726723
0.9948410553726723
[[  9137    840]
 [   840 161984]]


In [ ]:
mis = len(attack_data['label']) - len(label)

In [ ]:
label = np.append(label, np.ones(201))

In [ ]:
 len(attack_data['label'])

172801

In [84]:
wadi_pred = np.load('/content/drive/MyDrive/Colab_Notebooks/GTA/results/gta_WADI_ftM_sl60_ll30_pl1_nl3_dm124_nh8_el3_dl2_df124_atprob_ebfixed_test_0/pred.npy')
wadi_true = np.load('/content/drive/MyDrive/Colab_Notebooks/GTA/results/gta_WADI_ftM_sl60_ll30_pl1_nl3_dm124_nh8_el3_dl2_df124_atprob_ebfixed_test_0/true.npy')

In [85]:
import torch.nn as nn
wadi_pred = wadi_pred.squeeze()
wadi_true = wadi_true.squeeze()

In [202]:
wadi_pred = torch.from_numpy(wadi_pred)
wadi_true = torch.from_numpy(wadi_true)

# maxpool = nn.MaxPool1d(124)
# pred_mp = maxpool(wadi_pred)
# true_mp = maxpool(wadi_true)

In [224]:
from sklearn.metrics import mean_squared_error 
mse1 = wadi_true
for i in range(len(wadi_true)):
  mse1[i] = mean_squared_error(wadi_true[i], wadi_pred[i])

In [ ]:
# pd.DataFrame(label).to_csv('/content/drive/MyDrive/Colab_Notebooks/GTA/results/wadi_gta.csv')

----
### SWAT

In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
my_path = '/content/notebooks'
sys.path.append('/content/drive/MyDrive/Colab_Notebooks/GTA/myenv')
sys.path.append('/content/drive/MyDrive/Colab_Notebooks/GTA')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [214]:
import pandas as pd
import numpy as np
train = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/GTA/data/SWAT/SWaT_Dataset_Normal_v1_0.csv')
attack_data = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/GTA/data/SWAT/SWaT_Dataset_test.csv')

In [5]:
from datetime import datetime
import time
train[' Timestamp'] = pd.to_datetime(train[' Timestamp'])
attack_data[' Timestamp'] = pd.to_datetime(attack_data[' Timestamp'])
for i in range(len(train[' Timestamp'])):
  train[' Timestamp'].iloc[i] = time.mktime(train[' Timestamp'].iloc[i].timetuple())
for i in range(len(attack_data[' Timestamp'])):
  attack_data[' Timestamp'].iloc[i] = time.mktime(attack_data[' Timestamp'].iloc[i].timetuple())

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [16]:
train.to_csv('/content/drive/MyDrive/Colab_Notebooks/GTA/data/SWAT/SWaT_Dataset_Normal_v1_0.csv')
attack_data.to_csv('/content/drive/MyDrive/Colab_Notebooks/GTA/data/SWAT/SWaT_Dataset_test.csv')

In [14]:
train.drop('label', axis=1, inplace=True)

In [17]:
train1 = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/GTA/data/SWAT/SWaT_Dataset_Normal_v1_0.csv', index_col=[0])

In [20]:
%cd /content/drive/MyDrive/Colab_Notebooks/GTA

/content/drive/MyDrive/Colab_Notebooks/GTA


In [27]:
!python -u main_gta_dad.py --model gta --data SWaT --root_path '/content/drive/MyDrive/Colab_Notebooks/GTA/data/SWAT' --data_path 'SWaT_Dataset_Normal_v1_0.csv'

True
Use GPU: cuda:0
>>>>>>>start training : gta_SWaT_ftM_sl60_ll30_pl1_nl3_dm124_nh8_el3_dl2_df124_atprob_ebfixed_test_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 322228
val 112419
test 449859
	iters: 100, epoch: 1 | loss: 2136.4679275
	speed: 0.0839s/iter; left time: 802.8115s
	iters: 200, epoch: 1 | loss: 2109.4082040
	speed: 0.0796s/iter; left time: 753.4116s
	iters: 300, epoch: 1 | loss: 2082.3634169
	speed: 0.0804s/iter; left time: 752.9218s
	iters: 400, epoch: 1 | loss: 2055.3209757
	speed: 0.0798s/iter; left time: 739.1853s
	iters: 500, epoch: 1 | loss: 2028.2807609
	speed: 0.0801s/iter; left time: 734.2561s
	iters: 600, epoch: 1 | loss: 2001.2387096
	speed: 0.0792s/iter; left time: 718.3638s
	iters: 700, epoch: 1 | loss: 1974.1984742
	speed: 0.0803s/iter; left time: 719.9497s
	iters: 800, epoch: 1 | loss: 1947.1574995
	speed: 0.0795s/iter; left time: 704.6080s
	iters: 900, epoch: 1 | loss: 1920.1175767
	speed: 0.0805s/iter; left time: 706.1035s
	iters: 1000, epoch: 1 | loss: 1893.076746

In [35]:
pd.DataFrame(attack_data['label'])

,label
0,0
1,0
2,0
3,0
4,0
...,...
449914,0
449915,0
449916,0
449917,0


In [39]:
swat_label = np.load('/content/drive/MyDrive/Colab_Notebooks/GTA/results/gta_SWaT_ftM_sl60_ll30_pl1_nl3_dm124_nh8_el3_dl2_df124_atprob_ebfixed_test_1/label.npy')

In [41]:
swat_label.shape

(449919,)

In [40]:
swat_label = np.append(swat_label, np.zeros(119))

In [3]:
swat_pred = np.load('/content/drive/MyDrive/Colab_Notebooks/GTA/results/gta_SWaT_ftM_sl60_ll30_pl1_nl3_dm124_nh8_el3_dl2_df124_atprob_ebfixed_test_1/pred.npy')
swat_true = np.load('/content/drive/MyDrive/Colab_Notebooks/GTA/results/gta_SWaT_ftM_sl60_ll30_pl1_nl3_dm124_nh8_el3_dl2_df124_atprob_ebfixed_test_1/true.npy')

In [9]:
import torch
import torch.nn as nn
f = swat_true
for i in range(len(f)):
  f[i] = ((swat_pred[i] - swat_true[i])**2).sum()
f1 = torch.from_numpy(f)
mp1 = nn.MaxPool1d(52)
f1 = mp1(f1)
f1 = f1.squeeze()
f2 = f1.numpy()

In [13]:
f2

array([5.34728885, 5.3495517 , 5.35599079, ..., 6.05729641, 6.06201379,
       6.06239837])

In [96]:
f3 = np.where(f2 > 4.5, 0, 1)

In [98]:
pd.DataFrame(f3).value_counts()

0    415974
1     33826
dtype: int64

In [99]:
attack_data['label'].value_counts()

0    395298
1     54621
Name: label, dtype: int64

In [100]:
f3 = np.append(f3, np.zeros(119))

In [101]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
print(f1_score(attack_data['label'], f3))
print(recall_score(attack_data['label'], f3))
print(confusion_matrix(attack_data['label'], f3))

0.7620382828134363
0.6169788176708592
[[395172    126]
 [ 20921  33700]]


In [102]:
pd.DataFrame(f3).to_csv('/content/drive/MyDrive/Colab_Notebooks/GTA/results/swat_gta_se.csv')

---
### HAI21.03

In [108]:
from pathlib import Path
from datetime import *
import numpy as np
import os
import pandas as pd

def dataframe_from_csv(target):
    return pd.read_csv(target).rename(columns=lambda x: x.strip())

def dataframe_from_csvs(targets):
    return pd.concat([dataframe_from_csv(x) for x in targets])

In [109]:
hai_train_dataset = sorted([x for x in Path("/content/drive/MyDrive/Colab_Notebooks/HAI/data/New folder4/training").glob("*.csv")])
hai_train = dataframe_from_csvs(hai_train_dataset)

In [111]:
hai_test_dataset = sorted([x for x in Path("/content/drive/MyDrive/Colab_Notebooks/HAI/data/New folder4/testing").glob("*.csv")])
hai_test = dataframe_from_csvs(hai_test_dataset)

In [114]:
import time
hai_train[' Timestamp'] = pd.to_datetime(hai_train['time'])
hai_test[' Timestamp'] = pd.to_datetime(hai_test['time'])
for i in range(len(hai_train[' Timestamp'])):
  hai_train[' Timestamp'].iloc[i] = time.mktime(hai_train[' Timestamp'].iloc[i].timetuple())
for i in range(len(hai_test[' Timestamp'])):
  hai_test[' Timestamp'].iloc[i] = time.mktime(hai_test[' Timestamp'].iloc[i].timetuple())

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [115]:
hai_train.drop('time',axis=1, inplace = True)
hai_test.drop('time',axis=1, inplace = True)

In [118]:
hai_train.to_csv('/content/drive/MyDrive/Colab_Notebooks/GTA/data/HAI/HAI_train_total.csv')

In [ ]:
hai_test.to_csv('/content/drive/MyDrive/Colab_Notebooks/GTA/data/HAI/HAI_test_total.csv')

In [ ]:
%cd /content/drive/MyDrive/Colab_Notebooks/GTA
!python -u main_gta_dad.py --model gta --data HAI --root_path '/content/drive/MyDrive/Colab_Notebooks/GTA/data/HAI' --data_path 'HAI_train_total.csv'

/content/drive/MyDrive/Colab_Notebooks/GTA
True
Use GPU: cuda:0
>>>>>>>start training : gta_HAI_ftM_sl60_ll30_pl1_nl3_dm124_nh8_el3_dl2_df124_atprob_ebfixed_test_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 921543
val 89641
test 358744
	iters: 100, epoch: 1 | loss: 5056.6788605
	speed: 0.1462s/iter; left time: 3141.7474s
	iters: 200, epoch: 1 | loss: 4992.6672935
	speed: 0.1458s/iter; left time: 3118.5601s
	iters: 300, epoch: 1 | loss: 4928.6617873
	speed: 0.1459s/iter; left time: 3107.4124s
	iters: 400, epoch: 1 | loss: 4864.6574066
	speed: 0.1460s/iter; left time: 3093.7689s
	iters: 500, epoch: 1 | loss: 4800.6546189
	speed: 0.1452s/iter; left time: 3063.1096s
	iters: 600, epoch: 1 | loss: 4736.6528907
	speed: 0.1456s/iter; left time: 3057.6912s
	iters: 700, epoch: 1 | loss: 4672.6510297
	speed: 0.1458s/iter; left time: 3045.8512s
	iters: 800, epoch: 1 | loss: 4608.6501615
	speed: 0.1453s/iter; left time: 3021.5871s
	iters: 900, epoch: 1 | loss: 4544.6489105
	speed: 0.1456s/iter; left time: 301

In [20]:
import numpy as np
import pandas as pd

In [305]:
hai_train = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/GTA/data/HAI/HAI_train_total.csv', index_col=[0])

In [289]:
hai_pred = np.load('/content/drive/MyDrive/Colab_Notebooks/GTA/results/gta_HAI_ftM_sl60_ll30_pl1_nl3_dm124_nh8_el3_dl2_df124_atprob_ebfixed_test_0/pred.npy')
hai_true = np.load('/content/drive/MyDrive/Colab_Notebooks/GTA/results/gta_HAI_ftM_sl60_ll30_pl1_nl3_dm124_nh8_el3_dl2_df124_atprob_ebfixed_test_0/true.npy')

In [290]:
import torch
import torch.nn as nn
hai_f = hai_pred

In [291]:
for i in range(len(hai_true)):
  hai_f[i] = ((hai_pred[i] - hai_true[i])**2).sum()

In [302]:
hai_f

array([[[12.86742751, 12.86742751, 12.86742751, ..., 12.86742751,
         12.86742751, 12.86742751]],

       [[12.8138353 , 12.8138353 , 12.8138353 , ..., 12.8138353 ,
         12.8138353 , 12.8138353 ]],

       [[12.81535812, 12.81535812, 12.81535812, ..., 12.81535812,
         12.81535812, 12.81535812]],

       ...,

       [[ 6.50350192,  6.50350192,  6.50350192, ...,  6.50350192,
          6.50350192,  6.50350192]],

       [[ 6.52865291,  6.52865291,  6.52865291, ...,  6.52865291,
          6.52865291,  6.52865291]],

       [[ 6.70195999,  6.70195999,  6.70195999, ...,  6.70195999,
          6.70195999,  6.70195999]]])

In [294]:
f1 = torch.from_numpy(hai_f)
mp1 = nn.MaxPool1d(80)
f1 = f1.squeeze()
f1 = mp1(f1)
f1 = f1.squeeze()
f2 = f1.numpy()
f3 = np.where(f2 > 7, 0, 1)
f3 = np.append(f3, np.zeros(148))

In [295]:
pd.DataFrame(f3).value_counts()

0.0    335896
1.0     22908
dtype: int64

In [296]:
hai_test['attack'].value_counts()

0    350486
1      8318
Name: attack, dtype: int64

In [297]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
print(f1_score(hai_test['attack'], f3))
print(recall_score(hai_test['attack'], f3))
print(confusion_matrix(hai_test['attack'], f3))

0.018254019086658555
0.03426304400096177
[[327863  22623]
 [  8033    285]]
